### Set autoreloading
This extension will automatically update with any changes to packages in real time

In [ ]:
%load_ext autoreload
%autoreload 2

### Import packages
We'll need the `pynuml` and `nugraph` packages

In [ ]:
import pynuml
import nugraph as ng

### Open HDF5 event file

This file contains the low-level tables we need to process into graph objects

In [ ]:
file = pynuml.io.File('/raid/uboone/CHEP2023/CHEP2023.evt.h5')

### Create hit graph producer

Create a class that can read input information and use it to generate graphs

In [ ]:
processor = pynuml.process.HitGraphProducer(
        file=file,
        semantic_labeller=pynuml.labels.SimpleLabels(),
        event_labeller=pynuml.labels.FlavorLabels(),
        label_vertex=True)

## Configure plotting utility
Instantiate the **pynuml** utility for plotting graph objects, which will allow us to visualise the graphs we create

In [ ]:
plot = pynuml.plot.GraphPlot(
    planes=('u', 'v', 'y'),
    classes=('MIP', 'HIP', 'shower', 'michel', 'diffuse'))

### Load a buffer of input graphs

For interactive testing, we can use `read_data` to load the first 100 events in the event file using the following syntax, which will return a list of events that we can use to initialise an iterator

In [ ]:
file.read_data(0, 100)
evts = iter(file.build_evt())

### Retrieve the next event

This block retrieves an event from the buffer. Since we defined `evts` as an iterator over our 100 events, the following block can be executed multiple times, and each time it's executed, it will step to the next event in the buffer. After fetching an event, we pass it into the processor to create a graph. Not all events produce good graphs, so if the next event does not return a valid graph, the code will continue to fetch events until it finds the next event that produces a valid graph.

In [ ]:
name, data = processor(next(evts))
skip = 0
while data is None:
    name, data = processor(next(evts))
    skip += 1
if skip > 0:
    print(f'skipped {skip} events that did not produce valid graphs.')

### Visualise graph

We can use the plotting utility to take a closer look at the graph we just produced.

In [ ]:
fig = plot.plot(data, target='semantic', how='true', filter='true')
fig